# Lesson 2 - Building a QA Agent with Claude on Vertex AI

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access <code>requirements.txt</code> and the <code>data</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

In [ ]:
import base64
from pathlib import Path

from IPython.display import Markdown, display
from anthropic import AnthropicVertex
from anthropic.types import (
    Base64PDFSourceParam,
    DocumentBlockParam,
    MessageParam,
    TextBlockParam,
)
from dotenv import load_dotenv

from helpers import authenticate

In [ ]:
_ = load_dotenv()

In [ ]:
credentials, project_id = authenticate()

In [ ]:
client = AnthropicVertex(
    project_id=project_id,
    region="global",
    access_token=credentials.token,
)

In [ ]:
with Path("../data/2026AnthemgHIPSBC.pdf").open("rb") as file:
    pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

In [ ]:
prompt = "How much would I pay for mental health therapy?"

In [ ]:
response = client.messages.create(
    model="claude-haiku-4-5@20251001",
    max_tokens=1024,
    system="You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'",
    messages=[
        MessageParam(
            role="user",
            content=[
                DocumentBlockParam(
                    type="document",
                    source=Base64PDFSourceParam(
                        type="base64",
                        media_type="application/pdf",
                        data=pdf_data,
                    ),
                ),
                TextBlockParam(
                    type="text",
                    text=prompt,
                ),
            ],
        )
    ],
)

In [ ]:
response_text = response.content[0].text.replace("$", r"\$")
display(Markdown(response_text))

In [ ]:
%%writefile agents.py
import base64
from pathlib import Path

from anthropic import AnthropicVertex
from anthropic.types import (
    Base64PDFSourceParam,
    DocumentBlockParam,
    MessageParam,
    TextBlockParam,
)
from dotenv import load_dotenv

from helpers import authenticate

class PolicyAgent:
    def __init__(self) -> None:
        load_dotenv()
        credentials, project_id = authenticate()
        self.client = AnthropicVertex(
            project_id=project_id,
            region="global",
            access_token=credentials.token,
        )
        with Path("../data/2026AnthemgHIPSBC.pdf").open("rb") as file:
            self.pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

    def answer_query(self, prompt: str) -> str:
        response = self.client.messages.create(
            model="claude-haiku-4-5@20251001",
            max_tokens=1024,
            system="You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'",
            messages=[
                MessageParam(
                    role="user",
                    content=[
                        DocumentBlockParam(
                            type="document",
                            source=Base64PDFSourceParam(
                                type="base64",
                                media_type="application/pdf",
                                data=self.pdf_data,
                            ),
                        ),
                        TextBlockParam(
                            type="text",
                            text=prompt,
                        ),
                    ],
                )
            ],
        )
        
        return response.content[0].text.replace("$", r"\$")

In [ ]:
from agents import PolicyAgent

print("Running Health Insurance Policy Agent")
agent = PolicyAgent()
prompt = "How much would I pay for mental health therapy?"

response = agent.answer_query(prompt)
display(Markdown(response))